### Libraries

In [102]:
# Importing os.
import os

# Importing urllib.request.
import urllib.request

# Importing zipfile.
import zipfile

# Importing pandas.
import pandas as pd

# Importing numpy.
import numpy as np

# Importing train_test_split.
from sklearn.model_selection import train_test_split

# Importing pad_sequences.
from keras_preprocessing.sequence import pad_sequences

# Importing Sequential.
from keras.models import Sequential

# Importing Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation.
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation

# Importing Adam.
from keras.optimizers import Adam

# Importing backend.
from keras import backend

### Data Preprocessing

In [103]:
# Function used to download .zips.
def downloader(url, folder_name, filename):

  # Defining data folder path.
  data_path = os.path.join(os.getcwd(), folder_name)

  # Creating data folder.
  if not os.path.exists(data_path):
      os.makedirs(data_path)

  # Defining .zip file path.
  zip_path = os.path.join(os.getcwd(), folder_name, filename)

  # Requesting .zip file.
  if not os.path.exists(zip_path):
      urllib.request.urlretrieve(url, zip_path)

  # Extracting data from .zip.
  with zipfile.ZipFile(zip_path, "r") as zip_ref:
      zip_ref.extractall(path = data_path)

  # Returning data_path and zip_path.
  return data_path, zip_path

In [104]:
# Downloading dataset.
data_path, _ = downloader(url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip", folder_name = "data", filename = "dependency_treebank.zip")

# Downloading glove.
glove_path, _ = downloader(url = "https://nlp.stanford.edu/data/glove.6B.zip", folder_name = "glove", filename = "glove.6B.zip")

In [105]:
# Defining the dataset name.
dataset_name = "dependency_treebank"

# Defining path to first training sample.
file_path = os.path.join(data_path, dataset_name, "wsj_0001.dp")

# Reading first training sample.
if os.path.isfile(file_path):

  # Printing file.
  with open(file_path, mode = "r") as text_file: print(text_file.read())

Pierre	NNP	2
Vinken	NNP	8
,	,	2
61	CD	5
years	NNS	6
old	JJ	2
,	,	2
will	MD	0
join	VB	8
the	DT	11
board	NN	9
as	IN	9
a	DT	15
nonexecutive	JJ	15
director	NN	12
Nov.	NNP	9
29	CD	16
.	.	8

Mr.	NNP	2
Vinken	NNP	3
is	VBZ	0
chairman	NN	3
of	IN	4
Elsevier	NNP	7
N.V.	NNP	12
,	,	12
the	DT	12
Dutch	NNP	12
publishing	VBG	12
group	NN	5
.	.	3



In [106]:
# Defining embedding size.
EMBEDDING_SIZE = 50

# Defining specific glove's file path.
glove_file = os.path.join(os.getcwd(), glove_path, f"glove.6B.{str(EMBEDDING_SIZE)}d.txt")

# Reading lines of file.
with open(glove_file, encoding = "utf8" ) as text_file: 
  lines = text_file.readlines()

# Defining initial vocabulary.
embedding_vocabulary = {}

# Reading single lines.
for line in lines:

  # Splitting line.
  splits = line.split()

  # Storing line into vocabulary.
  embedding_vocabulary[splits[0]] = np.array([float(val) for val in splits[1:]])

# Printing one entry of the vocabulary.
print("The embedding for 'the' is:\n{}.".format(embedding_vocabulary["the"]))

The embedding for 'the' is:
[ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01].


In [107]:
# List containing dataframe rows.
dataframe_rows = []

# List containing words of a single sentence.
row_words = []

# List containing tags of a single sentence.
row_tags = []

# Defining data folder path.
folder = os.path.join(data_path, dataset_name)

# Storing rows.
for filename in sorted(os.listdir(folder)):

  # Computing path to file.
  file_path = os.path.join(folder, filename)

  # Checking existance of file.
  if os.path.isfile(file_path):

    # Opening the file.
    with open(file_path, mode = "r") as text_file:

      # Reading lines.
      while True:

        # Reading next line.
        line = text_file.readline()

        # Checking that line is different from "\n" (empty line) and from last line (EOF).
        if line and line != "\n":

          # Storing the word.
          row_words.append(line.split()[0])

          # Storing the POS tag.
          row_tags.append(line.split()[1])

        else:

          # Creating a row.
          dataframe_row = {"file_id": int(filename.split(".")[0].split("_")[1]), "sentence": row_words, "tags": row_tags}

          # Appending row.
          dataframe_rows.append(dataframe_row)

          # Resetting row_words list so to store a new sentence.
          row_words = []

          # Resetting row_tags list so to store a new sentence.
          row_tags = []

          # If, in particular, EOF is reached, then break the inner loop.
          if not line: break

# Creating pandas dataframe.
dataframe = pd.DataFrame(dataframe_rows)

# Printing dataframe head.
dataframe.head()

,file_id,sentence,tags
0,1,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ..."
1,1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ..."
2,2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP..."
3,3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS..."
4,3,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V..."


In [108]:
# Defining dataframe path.
dataframe_path = os.path.join(folder, dataset_name + ".pkl")

# Saving dataframe.
dataframe.to_pickle(dataframe_path)

In [109]:
# Defining training split.
TRAINING_SPLIT = range(1, 101)

# Defining validation split.
VALIDATION_SPLIT = range(101, 151)

# Defining test split.
TEST_SPLIT = range(151, 200)

# Computing train dataframe.
train = dataframe.loc[dataframe["file_id"].isin(TRAINING_SPLIT)]

# Computing validation dataframe.
validation = dataframe.loc[dataframe["file_id"].isin(VALIDATION_SPLIT)]

# Computing test dataframe.
test = dataframe.loc[dataframe["file_id"].isin(TEST_SPLIT)]

In [110]:
# Defining empty words and tags sets.
words, tags = set([]), set([])

# Creating set of words.
for sentence in train["sentence"].tolist():
  for word in sentence:
    words.add(word.lower())

# Creating set of tags
for tags_list in train["tags"].tolist():
  for tag in tags_list:
    tags.add(tag)

# Building vocabulary for words.
word_to_index = {word: i + 2 for i, word in enumerate(list(words))}
word_to_index["PAD"] = 0
word_to_index["OOV"] = 1

# Building vocabulary for tags.
tag_to_index = {tag: i + 1 for i, tag in enumerate(list(tags))}
tag_to_index["PAD"] = 0

In [111]:
word_to_index["the"]

623

In [112]:
# Function used to transform sentences into vectors of integers using the word_to_index vocabulary.
def convert_sentence(input_sentences, vocabulary):

  # Output sentences.
  output_sentences = []

  # Converting input sentences.
  for sentence in input_sentences:

    # List to store the words' indexes.
    sentence_indexes = []

    # Corventing each word in sentence.
    for word in sentence:

      try:

        # Computing index.
        sentence_indexes.append(vocabulary[word.lower()])

      except KeyError:

        # Computing index.
        sentence_indexes.append(vocabulary["OOV"])

    # Appending list of indexes to training set.
    output_sentences.append(sentence_indexes)

  # Returning output_sentences.
  return output_sentences

# Function used to transform list of tags into vectors of integers using the tag_to_index vocabulary.
def convert_tag(input_tags, vocabulary):

  # Output tags.
  output_tags = []

  # Converting input tags.
  for tags_list in input_tags:

    # Computing index.
    output_tags.append([vocabulary[tag] for tag in tags_list])

  # Returning output_tags.
  return output_tags

In [113]:
# Computing train sentences.
train_sentences = convert_sentence(input_sentences = train["sentence"].tolist(), vocabulary = word_to_index)

# Computing validation sentences.
validation_sentences = convert_sentence(input_sentences = validation["sentence"].tolist(), vocabulary = word_to_index)

# Computing test sentences.
test_sentences = convert_sentence(input_sentences = test["sentence"].tolist(), vocabulary = word_to_index)

# Computing train tags.
train_tags = convert_tag(input_tags = train["tags"].tolist(), vocabulary = tag_to_index)

# Computing validation tags.
validation_tags = convert_tag(input_tags = validation["tags"].tolist(), vocabulary = tag_to_index)

# Computing test tags.
test_tags = convert_tag(input_tags = test["tags"].tolist(), vocabulary = tag_to_index)

In [114]:
print(train_sentences[0])
print(train_tags[0])

[6467, 6250, 2556, 3738, 4892, 2373, 2556, 927, 5216, 623, 5120, 3592, 3139, 6876, 2763, 3920, 4950, 7225]
[39, 39, 7, 6, 24, 38, 7, 36, 13, 41, 22, 11, 41, 38, 22, 39, 6, 23]


In [115]:
# Computing length of longest train sentence.
MAX_LENGTH = len(max(train["sentence"].tolist(), key = len))

# Padding sets.
train_sentences = pad_sequences(train_sentences, maxlen = MAX_LENGTH, padding = "post")
validation_sentences = pad_sequences(validation_sentences, maxlen = MAX_LENGTH, padding = "post")
test_sentences = pad_sequences(test_sentences, maxlen = MAX_LENGTH, padding = "post")
train_tags = pad_sequences(train_tags, maxlen = MAX_LENGTH, padding = "post")
validation_tags = pad_sequences(validation_tags, maxlen = MAX_LENGTH, padding = "post")
test_tags = pad_sequences(test_tags, maxlen = MAX_LENGTH, padding = "post")

In [116]:
# Number of converted words.
hits = 0

# Number of OOV words.
misses = 0

# Preparing embedding matrix so that OOV words will be assigned a vector of zeros.
embedding_matrix = np.zeros((len(word_to_index), int(EMBEDDING_SIZE)))

# Retrieving embedding vector for each word.
for word, i in word_to_index.items():

  # Computing vector.
  vector = embedding_vocabulary.get(word)

  # Checking vector.
  if vector is not None:
    
    # Populating the embedding matrix.
    embedding_matrix[i] = vector

    # Increasing number of hits.
    hits += 1

  else:

    # Increasing number of misses.
    misses += 1

# Printing number of hits and missed.
print(f"Converted {hits} words. Missed {misses} words.")

Converted 7045 words. Missed 361 words.


In [117]:
embedding_matrix[word_to_index["the"]]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [118]:
# Function used to one hot encode tag lists.
def one_hot_encoder(tags_lists, n_categories):

  # List of encodings.
  one_hot_encodings = []

  # Iterating over lists of tags (e.g. train_tags).
  for tags_list in tags_lists:

    # One-hot encoding of the specific list.
    one_hot_encoding = []

    # Iterating over tags
    for tag in tags_list:

      # Creating vector of zeros.
      one_hot_encoding.append(np.zeros(n_categories))

      # Storing a one in the tag position.
      one_hot_encoding[-1][tag] = 1.0

    # Appending the one hot encoding.
    one_hot_encodings.append(one_hot_encoding)
  
  # Returning the list of encodings.
  return np.array(one_hot_encodings)

# Function used to evaluate performance of models.
def custom_accuracy(tag_to_ignore):

  # Function used to compute new accuracy, ignoring specific classes.
  def ignore_tags(y_true, y_pred):

    # Computing true and predicted classes.
    y_true_class = backend.argmax(y_true, axis = -1)
    y_pred_class = backend.argmax(y_pred, axis = -1)

    # Computing mask to not consider tag to ignore.
    mask = backend.cast(backend.not_equal(y_pred_class, tag_to_ignore), "int32")

    # Computing matches without considering tags to ignore.
    matches = backend.cast(backend.equal(y_true_class, y_pred_class), "int32") * mask

    # Computing custom accuracy without considering tags to ignore.
    accuracy = backend.sum(matches) / backend.maximum(backend.sum(mask), 1)

    # Returning custom accuracy.
    return accuracy
  
  # Returning ignore_class_accuracy function.
  return ignore_tags

In [119]:
# One-hot encoding train tags.
train_tags = one_hot_encoder(tags_lists = train_tags, n_categories = len(tag_to_index))

# One-hot encoding validation tags.
validation_tags = one_hot_encoder(tags_lists = validation_tags, n_categories = len(tag_to_index))

# One-hot encoding test tags.
test_tags = one_hot_encoder(tags_lists = test_tags, n_categories = len(tag_to_index))

### Model Definition

In [120]:
# Model sketch.
model = Sequential()
model.add(InputLayer(input_shape = (MAX_LENGTH,))) 
model.add(Embedding(len(word_to_index), EMBEDDING_SIZE, weights = [embedding_matrix], trainable = False))
model.add(Bidirectional(LSTM(256, return_sequences = True)))
model.add(TimeDistributed(Dense(len(tag_to_index))))
model.add(Activation("softmax"))

# Compiling.
model.compile(loss = "categorical_crossentropy", optimizer = Adam(), metrics = ["accuracy", custom_accuracy(word_to_index["PAD"])])

# Printing summary.
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 249, 50)           370300    
                                                                 
 bidirectional_7 (Bidirectio  (None, 249, 512)         628736    
 nal)                                                            
                                                                 
 time_distributed_7 (TimeDis  (None, 249, 46)          23598     
 tributed)                                                       
                                                                 
 activation_7 (Activation)   (None, 249, 46)           0         
                                                                 
Total params: 1,022,634
Trainable params: 652,334
Non-trainable params: 370,300
_________________________________________________________________


In [121]:
# Fitting the model.
model.fit(train_sentences, train_tags, batch_size = 256, epochs = 40, validation_data = (validation_sentences, validation_tags))

Epoch 1/40
8/8 [==============================] - 5s 277ms/step - loss: 2.7618 - accuracy: 0.8627 - ignore_tags: 0.1120 - val_loss: 0.4893 - val_accuracy: 0.9091 - val_ignore_tags: 0.1669
Epoch 2/40
8/8 [==============================] - 1s 154ms/step - loss: 0.3780 - accuracy: 0.9148 - ignore_tags: 0.1874 - val_loss: 0.3224 - val_accuracy: 0.9190 - val_ignore_tags: 0.2008
Epoch 3/40
8/8 [==============================] - 1s 151ms/step - loss: 0.3004 - accuracy: 0.9207 - ignore_tags: 0.2125 - val_loss: 0.2852 - val_accuracy: 0.9220 - val_ignore_tags: 0.2262
Epoch 4/40
8/8 [==============================] - 1s 153ms/step - loss: 0.2719 - accuracy: 0.9263 - ignore_tags: 0.2642 - val_loss: 0.2657 - val_accuracy: 0.9277 - val_ignore_tags: 0.2713
Epoch 5/40
8/8 [==============================] - 1s 155ms/step - loss: 0.2548 - accuracy: 0.9300 - ignore_tags: 0.2948 - val_loss: 0.2513 - val_accuracy: 0.9316 - val_ignore_tags: 0.3145
Epoch 6/40
8/8 [==============================] - 1s 154ms/s

In [122]:
print(f"Accuracy on test set: {model.evaluate(test_sentences, test_tags)[-1] * 100}%.")

21/21 [==============================] - 0s 14ms/step - loss: 0.0730 - accuracy: 0.9782 - ignore_tags: 0.7722
Accuracy on test set: 77.22469568252563%.


In [123]:
# TODO: compute macro F1-score.